<a href="https://colab.research.google.com/github/sahaanirbannew/spacy-custom-NER-bird-names/blob/main/Program_fetch_tweet_based_on_search_term%2C_create_training_data%2C_ML_model_to_NER_bird_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Function definitions

In [6]:
def what_search_term():
  search_term = "indiAves"        ## CHANGE THIS. 
  return search_term
  

import tweepy
import time
import pickle
import csv
import datetime 
from datetime import datetime 
import os, sys
from google.colab import drive
import pandas as pd 
import re
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans
import shutil
from zipfile import ZipFile
import tensorflow as tf

def connect_to_google_drive():
  drive.mount("/content/drive", force_remount=False)

def extract_model_best_from_archive():
  file_name = "/content/model-best.zip"
  with ZipFile(file_name, 'r') as zip:
    zip.extractall("/content/model-best")

def is_GPU_on():
  if tf.test.gpu_device_name() == "": 
    return False
  return True 

def create_twitter_app_obj():
  consumer_key = "iPaIdR8GRI59yTJMs0Es0dIBN"
  consumer_secret = "pLadg3UaLeK3yKDujRMChRN3p8hUDBOjBsuOBy8j8ERr4zz1vs"
  access_token = "39085479-AabHt6bmFSbClDfUZuHjModYPAxVlOxHeMA79UyVt"
  access_token_secret = "3IqXDISfqg14wzMNNn2AX4KYG9Wfkltt21QxKasE4YNnG"
  # Creating the authentication object
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  # Setting your access token and secret
  auth.set_access_token(access_token, access_token_secret)
  # Creating the API object while passing in auth information

  try:
    api = tweepy.API(auth) 
    return api
  except:
    print("Error: Error making the Twitter Object.")
    return 0

def get_google_drive_folder_path():
  path = "/content/drive/My Drive/IndiAves/" 
  return path 

def get_since_id(path,search_word):
  file = open(path+"since_id",'rb')
  try:
    since_id_data = pickle.load(file)
  except Exception as e:
    print(str(e))
    return 0
  try: 
    since_id = since_id_data[search_word]
    return since_id
  except:
    print("Info: This looks like a new search.")
    return 0

def load_all_birds_list(path):
  file = open(path+"bird_list_df",'rb')
  bird_list_df = pickle.load(file)
  try: 
    return bird_list_df
  except:
    print("Error: No bird list found.")
    return 0
  
def load_training_data(path):
  file = open(path+"training_data",'rb')
  training_data = pickle.load(file)
  try: 
    return training_data
  except:
    print("Error: No training data found.")
    return 0

def load_nlp_model_():
  import spacy
  #nlp = spacy.load("en_core_web_sm")
  nlp = spacy.blank("en")
  print(nlp.pipe_names)
  return nlp

def create_doc_bin(training_data,nlp):
  from spacy.tokens import DocBin
  from tqdm import tqdm
  from spacy.util import filter_spans
  doc_bin = DocBin()
  for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        #if span is None:
        #    print("Skipping entity")
        #else:
        #    ents.append(span)
        if span is not None:
          ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)
  doc_bin.to_disk("training_data.spacy") # save the docbin object

def export_training_data(training_data,path):
  filepath = open(path+"training_data",'wb') 
  pickle.dump(training_data, filepath)                     
  filepath.close() 
  print(len(training_data["annotations"]), "records in training data.")
  print("training data is saved.") 


def initialisation():
  !pip install tweet-preprocessor
  !python -m spacy download en_core_web_sm
  !pip install spacy[transformers]

def import_base_config_file(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"base_config.cfg", "/content/base_config.cfg")

def basic_preprocess(tweet):
  import preprocessor as p
  p.set_options(p.OPT.EMOJI, p.OPT.MENTION, p.OPT.URL, p.OPT.SMILEY, p.OPT.NUMBER,p.OPT.HASHTAG)
  tweet = tweet.lower()
  tweet = tweet.replace("\n"," ")  
  tweet = tweet.replace("\\n"," ")
  tweet = tweet.replace("'","")
  tweet = tweet[1:] 
  tweet = p.clean(tweet)
  tweet = re.sub(r'[^\w\s]', ' ', tweet)
  tweet = re.sub(r' x..', '', tweet)
  tweet = re.sub(r' +', ' ', tweet) #' +', ' '
  #tweet = re.sub(r' n. ', '', tweet) 
  tweet = tweet.replace("x9c","")
  tweet = tweet.strip()
  return tweet

def preprocess_tweets_arr_batch(tweets):
  new_tweets_ = []
  for tweet in tweets:
    tweet = basic_preprocess(tweet)
    if len(tweet) > 1:
      new_tweets_.append(tweet)
  return new_tweets_

def add_to_database(api, search_word, ext_path, since_id):
  new_search = "#" +search_word + " -filter:retweets" 
  count = 0
  csvFile = open(ext_path+search_word+'.csv', 'a')
  csvFile2 = open('/content/temp_.csv', 'w') 

  csvWriter = csv.writer(csvFile)
  csvWriter2 = csv.writer(csvFile2)

  for tweet in tweepy.Cursor(api.search,q=new_search,count=100,
                            lang="en",
                            since_id=since_id, tweet_mode="extended").items(): 
    if tweet.id > since_id:
      since_id = tweet.id
    
    media_url=""
    if tweet.entities.get('media', []): media_url = tweet.entities.get('media', [])[0]['media_url']

    hashtags = []
    for tag in tweet.entities["hashtags"]:
      hashtags.append(tag["text"]) 
    
    csvWriter.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    csvWriter2.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    count +=1
  print(datetime.now(),count, "tweets retrieved.")
  return since_id 

def update_since_id(since_id, path,search_word):
  filepath = open(path+"since_id",'rb')
  since_id_data = pickle.load(filepath)
  since_id_data[search_word]=since_id
  filepath = open(path+"since_id",'wb') 
  pickle.dump(since_id_data, filepath)                     
  filepath.close()

def train_model():
  !python -m spacy init fill-config base_config.cfg config.cfg
  !python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

def import_best_model(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"model-best.zip", "/content/model-best.zip")

def archive_best_model_in_google_drive(google_folder_path):
  shutil.make_archive(google_folder_path+"model-best", 'zip', "/content/model-best")

def train_and_load_best_model(training_data,nlp,google_drive_folder_path): 
  is_nlp_ready = True 

  try:
    is_GPU_on_colab = is_GPU_on() 
    print("is_GPU_on",is_GPU_on_colab)
    if is_GPU_on_colab == True: 
      create_doc_bin(training_data,nlp)

      #train the model 
      try:
        import_base_config_file(google_drive_folder_path)
        print("Base Config File fetched.")
      except Exception as e: 
        print(str(e))
        print("Error: Base Config File is NOT fetched.")

      #trains the model.
      try:
        train_model()
        #archive the best model to Google Drive. 
        archive_best_model_in_google_drive(google_drive_folder_path)
      except Exception as e:
        print(str(e))
        print("Error: Model training failed.")
        is_GPU_on_colab = False #so that it fetches the last saved model. 
    
    if is_GPU_on_colab == False: 
      import_best_model(google_drive_folder_path)
      extract_model_best_from_archive()
    
    #load the best model
    nlp_ner = spacy.load("model-best") 
  except Exception as e:
    return False, ""

  return is_nlp_ready, nlp_ner


def load_best_model_from_archive(google_drive_folder_path):
  import_best_model(google_drive_folder_path)
  extract_model_best_from_archive()
  nlp_ner = spacy.load("model-best") 
  return True, nlp_ner

def update_training_data(training_data,new_tweets,all_birds_name,is_nlp_ready,nlp_ner):
  for tweet in new_tweets:
    tweet_bird_found = False 
    print("------------***********") 
    print(tweet) 
    print("-Labelling work:")
    for bird in all_birds_name:
      if tweet.find(bird.strip())>-1:
        print("--From rule matching:")
        print(bird, tweet.find(bird), tweet.find(bird)+len(bird))
        training_data["annotations"].append({'text': tweet, 'entities': [(tweet.find(bird), tweet.find(bird)+len(bird), 'BIRDNAME')]})
        tweet_bird_found = True  
        
    
    if tweet_bird_found == False and is_nlp_ready == True:
        doc = nlp_ner(tweet)
        suggestion = str(doc.ents).replace("(","").replace(")","").replace(",","")
        print("--Suggestion from NER model: "+suggestion)
        birds_input = input("Birds,:").split(",")
        if birds_input != "-":
          for bird_ in birds_input: 
            bird_ = bird_.strip()
            print(bird_, tweet.find(bird_),tweet.find(bird_)+len(bird_))
            training_data["annotations"].append({'text': tweet, 'entities': [(tweet.find(bird_), tweet.find(bird_)+len(bird_), 'BIRDNAME')]})
  return training_data 


# Main Program - 1

Run this only once.

Note:


*   Please use GPU to train model
*   If GPU is not available, then last best model would be loaded. 
*   Assumption: All files would be available in the folder 




In [7]:
initialisation() 
import spacy 

#what is the search term?
search_word = what_search_term() 
print(search_word)

#connect to google drive
connect_to_google_drive()

#connect to twitter api. 
twitter_api_obj = create_twitter_app_obj() 

#get Google Drive folder path
google_drive_folder_path = get_google_drive_folder_path() 
print(google_drive_folder_path)

#get since_id based on the search term
since_id = get_since_id(google_drive_folder_path,search_word) 
#print(since_id)

#load all birds list 
all_birds = load_all_birds_list(google_drive_folder_path)

#load training data
training_data = load_training_data(google_drive_folder_path) 

#Set up Model Training. 
# PS: GPU needed. 
nlp = load_nlp_model_() #this is spacy stuff. 

is_nlp_ready,nlp_ner = train_and_load_best_model(training_data,nlp,google_drive_folder_path)
is_nlp_ready,nlp_ner = load_best_model_from_archive(google_drive_folder_path) #for quicker work

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 31.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
indiAves
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/IndiAves/
[]
is_GPU_on True


100%|██████████| 2100/2100 [00:00<00:00, 4212.45it/s]


Base Config File fetched.
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-09-01 02:17:43,404] [INFO] Set up nlp object from config
[2022-09-01 02:17:43,415] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-09-01 02:17:43,419] [INFO] Created vocabulary
[2022-09-01 02:17:43,420] [INFO] Finished initializing nlp object
[2022-09-01 02:17:54,446] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.0

# Main Program - 2

Keep re-running this!

In [9]:
#fetch tweets
since_id = add_to_database(twitter_api_obj,search_word, google_drive_folder_path, since_id) 
new_tweets_df = pd.read_csv("/content/temp_.csv", names=["created_at", "tweet_id", "user", "location", "tweet", "media_url", "hashtags"])
new_tweets = preprocess_tweets_arr_batch(new_tweets_df["tweet"].tolist())
all_birds_name = all_birds["bird_name"].tolist() 

#Update training data. ###NEEDS USER INTERVENTION. 
update_training_data(training_data,new_tweets,all_birds_name,is_nlp_ready,nlp_ner) 

#Exports training data 
export_training_data(training_data,google_drive_folder_path)

#Update since_id
update_since_id(since_id,google_drive_folder_path,search_word)

#delete temp_.csv 
os.remove("/content/temp_.csv")

#copy the training data to baba's folder so that I can train the model.
shutil.copyfile("/content/drive/My Drive/IndiAves/training_data", "/content/drive/My Drive/IndiAves_baba/training_data")

2022-09-01 02:28:11.557147 0 tweets retrieved.
2102 records in training data.
training data is saved.


'/content/drive/My Drive/IndiAves_baba/training_data'

# Stand alone test.

In [10]:
sentence = "this is a preprocessed text and it should find spotted dove"
doc = nlp_ner(sentence)
print(doc.ents)

(spotted dove,)


**Labelling yesterdays data**